In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17176259
paper_name = 'bishop_avery_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['hits_genenames_resistant.txt','hits_genenames_sensitive.txt']

In [6]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/'+f, sep='\t', header=None)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['gene'] = original_data[0].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = np.power(-1, ixf)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    original_data_list.append(original_data)

Original data dimensions: 45 x 1
       0
0   HSL7
1   SIT4
2   DOA4
3  VPS24
4   VPS1
Empty DataFrame
Columns: [0, gene, orf]
Index: []
(34, 1)
Original data dimensions: 47 x 1
      0
0  CYC8
1  DOT6
2  GLO3
3  SSN8
4  EAF7
Empty DataFrame
Columns: [0, gene, orf]
Index: []
(47, 1)


In [7]:
original_data = pd.concat(original_data_list, axis=0)

In [8]:
original_data.shape

(81, 1)

In [9]:
original_data.head()

,data
orf,
YBR119W,1
YBR120C,1
YBR132C,1
YBR133C,1
YBR145W,1


In [10]:
original_data = original_data.groupby(original_data.index).mean()

In [11]:
original_data.shape

(81, 1)

# Load & process tested strains

In [12]:
# tested1 = pd.read_excel('raw_data/Mata_DeletionArray+slow_growers.xlsx', sheet_name='96', skiprows=1)
# tested2 = pd.read_excel('raw_data/Mata_DeletionArray+slow_growers.xlsx', sheet_name='slow growers', skiprows=1)

In [13]:
# tested1.head()

In [14]:
# tested = pd.concat([tested1[['ORF ']].rename(columns={'ORF ': 'ORF'}), tested2[['ORF']]], axis=0)

In [15]:
# tested['orf'] = tested['ORF'].astype(str)

In [16]:
# tested['orf'] = clean_orf(tested['orf'])

In [17]:
# tested['orf'] = translate_sc(tested['orf'].values, to='orf')

In [18]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,])

In [19]:
# tested_orfs = tested['orf'].unique()

In [20]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [21]:
# tested_orfs = list(tested_orfs) + missing

In [22]:
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [173]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,173
data_type,value
orf,
YAL035W,-1
YAL042W,-1
YBL099W,-1
YBL100C,-1
YBR112C,-1


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,173
,data_type,value
gene_id,orf,
33,YAL035W,-1
40,YAL042W,-1
187,YBL099W,-1
188,YBL100C,-1
306,YBR112C,-1


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        173          
data_type       value    valuez
gene_id orf                    
33      YAL035W    -1 -7.414789
40      YAL042W    -1 -7.414789
187     YBL099W    -1 -7.414789
188     YBL100C    -1 -7.414789
306     YBR112C    -1 -7.414789

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
# from IO.save_data_to_db3 import *

In [37]:
# save_data_to_db(data_all, paper_pmid)